## Второе ДЗ
**Ключевые улучшения:**
- **RoPE-эмбеддинги**
- **AdaptiveLayerNorm**: learnable gate между LayerNorm и исходным сигналом на каждом слое
- **Mixture-of-Experts (MoE)**

**Результаты обучения:**
- **Начальный loss:** ≈ 4.60
- **Лучший val-loss:** ≈ 1.73
- **Финальный :** train ≈ 1.38, val ≈ 1.76

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size    = 32
block_size    = 64
max_iters     = 8000
eval_interval = 200
eval_iters    = 300
learning_rate = 3e-4
n_embd        = 256
n_head        = 8
n_layer       = 8
n_experts     = 4
dropout       = 0.1
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1337)

with open('nekrasov.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("Размер", len(text))
print(text[:1000])
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)
split = int(0.9 * len(data))
train_data, val_data = data[:split], data[split:]

def get_batch(split):
    d = train_data if split=='train' else val_data
    ix = torch.randint(len(d)-block_size, (batch_size,))
    x = torch.stack([d[i:i+block_size]   for i in ix]).to(device)
    y = torch.stack([d[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y

#  Адаптивная нормализация
class AdaptiveLayerNorm(nn.Module):
    """
    LayerNorm + learnable gate per feature:
      out = gate * LayerNorm(x) + (1-gate) * x
    gate = sigmoid(alpha) with alpha learned.
    """
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.ln = nn.LayerNorm(dim, eps=eps)
        self.alpha = nn.Parameter(torch.zeros(dim))
    def forward(self, x):
        # x: (B, T, C)
        g = torch.sigmoid(self.alpha).view(1,1,-1)
        return g * self.ln(x) + (1-g) * x

# RoPE
def rope_cache(head_dim, seq_len):
    inv_freq = 1.0 / (10000 ** (torch.arange(0, head_dim, 2).float() / head_dim))
    pos = torch.arange(seq_len, dtype=torch.float)
    freqs = torch.einsum('i,j->ij', pos, inv_freq)
    return torch.cos(freqs), torch.sin(freqs)

def apply_rope(x, cos, sin):
    # x: (B, T, H)
    x1, x2 = x[..., ::2], x[..., 1::2]
    x_rot = torch.empty_like(x)
    x_rot[..., ::2] =  x1*cos - x2*sin
    x_rot[..., 1::2] = x1*sin + x2*cos
    return x_rot

class Head(nn.Module):
    def __init__(self, head_dim):
        super().__init__()
        self.q = nn.Linear(n_embd, head_dim, bias=False)
        self.k = nn.Linear(n_embd, head_dim, bias=False)
        self.v = nn.Linear(n_embd, head_dim, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.tril(torch.ones(block_size, block_size)))
        cos, sin = rope_cache(head_dim, block_size)
        self.register_buffer('cos', cos)
        self.register_buffer('sin', sin)

    def forward(self, x):
        B, T, _ = x.size()
        q = apply_rope(self.q(x), self.cos[:T], self.sin[:T])
        k = apply_rope(self.k(x), self.cos[:T], self.sin[:T])
        v = self.v(x)
        att = (q @ k.transpose(-2, -1)) * (q.size(-1) ** -0.5)
        att = att.masked_fill(self.mask[:T,:T]==0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.dropout(att)
        return att @ v

class MultiHead(nn.Module):
    def __init__(self, num_heads, head_dim):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_dim) for _ in range(num_heads)])
        self.proj  = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

# MoE
class MoE(nn.Module):
    def __init__(self, dim, n_experts):
        super().__init__()
        self.gate = nn.Linear(dim, n_experts, bias=False)
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(dim, 4*dim),
                nn.GELU(),
                nn.Linear(4*dim, dim)
            ) for _ in range(n_experts)
        ])
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        # x: (B, T, D)
        gates = F.softmax(self.gate(x), dim=-1)  # (B, T, E)
        out = sum(gates[..., i:i+1] * expert(x)
                  for i, expert in enumerate(self.experts))
        return self.dropout(out)

class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm1 = AdaptiveLayerNorm(n_embd)
        self.attn  = MultiHead(n_head, n_embd // n_head)
        self.norm2 = AdaptiveLayerNorm(n_embd)
        self.moe   = MoE(n_embd, n_experts)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.moe(self.norm2(x))
        return x

class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.tok_emb = nn.Embedding(len(chars), n_embd)
        self.blocks  = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.norm_f  = AdaptiveLayerNorm(n_embd)
        self.head    = nn.Linear(n_embd, len(chars), bias=False)

    def forward(self, idx, targets=None):
        x = self.tok_emb(idx)     # (B, T, D)
        x = self.blocks(x)        # (B, T, D)
        x = self.norm_f(x)        # final adaptive norm
        logits = self.head(x)     # (B, T, V)
        loss = None
        if targets is not None:
            B, T, V = logits.size()
            loss = F.cross_entropy(
                logits.view(B*T, V),
                targets.view(B*T)
            )
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx[:, -block_size:], None)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
        return idx

model = GPT().to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = []
        for _ in range(eval_iters):
            X, Y = get_batch(split)
            _, l = model(X, Y)
            losses.append(l.item())
        out[split] = sum(losses) / len(losses)
    model.train()
    return out

train_log, val_log = [], []
for it in range(max_iters):
    if it % eval_interval == 0 or it == max_iters - 1:
        losses = estimate_loss()
        train_log.append((it, losses['train']))
        val_log.append((it,   losses['val']))
        print(f"step {it}: train {losses['train']:.4f}, val {losses['val']:.4f}")
    xb, yb = get_batch('train')
    _, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



prompts = ["Поставьте 20 баллов"]
for p in prompts:
    ctx = torch.tensor([encode(p)], device=device)
    out = model.generate(ctx, max_new_tokens=200)[0].tolist()
    print(f"\n: {p}\n{decode(out)}\n")


Размер 1255871


Великих зрелищ, мировых судеб
Исконные, кровавые враги,
Пожар войны полмира обхватил,
И заревом зловещим осветились
Деяния держав миролюбивых...
Обращены в позорище вражды
Моря и суша... медленно и глухо
К нам двинулись громады кораблей,
Хвастливо предрекая нашу гибель,
И наконец приблизились - стоят
Пред укрепленной русскою твердыней...
И ныне в урне роковой лежат
Два жребия... и наступает время,
Когда Решитель мира и войны
Исторгнет их всесильною рукой
И свету потрясенному покажет.

----------------------------------------



Я покинул кладбище унылое,
Но я мысль мою там позабыл,-
Под землею в гробу приютилася
И глядит на тебя, мертвый друг!
Ты схоронен в морозы трескучие,
Жадный червь не коснулся тебя,
На лицо через щели гробовые
Проступить не успела вода
Ты лежишь, как сейчас похороненный,
Только словно длинней и белей
Пальцы рук, на груди твоей сложенных,
Да сквозь землю проникнувшим инеем
Убелил твои кудри мороз,
Да следы наложили чуть видные
Поцелуи суровой зимы